In [10]:
from lohrasb.best_estimator import BaseModel
from optuna.pruners import HyperbandPruner
from optuna.samplers._tpe.sampler import TPESampler
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import optuna
from sklearn.pipeline import Pipeline
from feature_engine.imputation import (
    CategoricalImputer,
    MeanMedianImputer
    )
from category_encoders import OrdinalEncoder
from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    f1_score)
from lightgbm import *
from lohrasb.utils.metrics import f1_plus_tn


#### Example 1 : Use Adult Data Set (a classification problem)
  
https://archive.ics.uci.edu/ml/datasets/Adult

#### Part 1: Use BestModel in sklearn pipeline


In [11]:
urldata= "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data"
# column names
col_names=["age", "workclass", "fnlwgt" , "education" ,"education-num",
"marital-status","occupation","relationship","race","sex","capital-gain","capital-loss","hours-per-week",
"native-country","label"
]
# read data
data = pd.read_csv(urldata,header=None,names=col_names,sep=',')
data.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,label
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


#### Define labels


In [12]:
data.loc[data['label']=='<=50K','label']=0
data.loc[data['label']==' <=50K','label']=0

data.loc[data['label']=='>50K','label']=1
data.loc[data['label']==' >50K','label']=1

data['label']=data['label'].astype(int)

#### Train test split

In [13]:
X = data.loc[:, data.columns != "label"]
y = data.loc[:, data.columns == "label"]
y = y.values.ravel()


X_train, X_test, y_train, y_test =train_test_split(X, y, test_size=0.33, random_state=42)

# for sample_weights
weights = np.ones(len(y_train))


#### Find feature types for later use

In [14]:
int_cols =  X_train.select_dtypes(include=['int']).columns.tolist()
float_cols =  X_train.select_dtypes(include=['float']).columns.tolist()
cat_cols =  X_train.select_dtypes(include=['object']).columns.tolist()


#### Define model and set it argumens 

In [15]:
estimator = LGBMClassifier()
estimator_params = {
        "boosting_type":["gbdt"],
        "max_depth": [6,15],
        "learning_rate":[0.01, 0.1],
        "random_state":[42],

    }

kwargs = {  # params for fit method  
            'fit_optuna_kwargs' :{
            'sample_weight':None,
            },
            # params for OptunaSearch
            'main_optuna_kwargs' : {
            'estimator':estimator,
            'estimator_params':estimator_params,
            'refit':True,
            'measure_of_accuracy' :'f1_score(y_true, y_pred,average="weighted")',

            },
            'train_test_split_kwargs':{
                'test_size':.3,
                            
            },
            'study_search_kwargs':{
                'storage':None,
                'sampler':TPESampler(),
                'pruner':HyperbandPruner(),
                'study_name':"example of optuna optimizer",
                'direction':"maximize",
                'load_if_exists':False,
            },
            'optimize_kwargs':{
                # optuna optimization params
                'n_trials':20,
                'timeout':600,
                'catch':(),
                'callbacks':None,
                'gc_after_trial':False,
                'show_progress_bar':False,
            }
}




In [16]:
obj = BaseModel().optimize_by_optuna(
            kwargs=kwargs
        )

#### Build sklearn pipeline

In [17]:


pipeline =Pipeline([
            # int missing values imputers
            ('intimputer', MeanMedianImputer(
                imputation_method='median', variables=int_cols)),
            # category missing values imputers
            ('catimputer', CategoricalImputer(variables=cat_cols)),
            #
            ('catencoder', OrdinalEncoder()),
            # classification model
            ('obj', obj),


 ])
 



#### Run Pipeline

In [18]:
pipeline.fit(X_train,y_train)
y_preds = pipeline.predict(X_test)
pred_labels = np.rint(y_preds)




[I 2023-07-02 12:51:11,425] A new study created in memory with name: example of optuna optimizer
[I 2023-07-02 12:51:12,884] Trial 0 finished with value: 0.8708378541567025 and parameters: {'boosting_type': 'gbdt', 'max_depth': 6, 'learning_rate': 0.08474901727393494, 'random_state': 42}. Best is trial 0 with value: 0.8708378541567025.
[I 2023-07-02 12:51:13,647] Trial 1 finished with value: 0.852582531256088 and parameters: {'boosting_type': 'gbdt', 'max_depth': 14, 'learning_rate': 0.013285676707899145, 'random_state': 42}. Best is trial 0 with value: 0.8708378541567025.
[I 2023-07-02 12:51:14,399] Trial 2 finished with value: 0.8701739258494435 and parameters: {'boosting_type': 'gbdt', 'max_depth': 13, 'learning_rate': 0.07001673245222276, 'random_state': 42}. Best is trial 0 with value: 0.8708378541567025.
[I 2023-07-02 12:51:15,079] Trial 3 finished with value: 0.8675783655603083 and parameters: {'boosting_type': 'gbdt', 'max_depth': 7, 'learning_rate': 0.05254214418602758, 'rando

2023-07-02 12:51:26,953 :: dev :: If refit is set to True, the optimal model will be refit on the entire dataset, i.e., X_train and y_train!
2023-07-02 12:51:26,953 :: dev :: If refit is set to True, the optimal model will be refit on the entire dataset, i.e., X_train and y_train!


#### Check performance of the pipeline

In [19]:
print('F1 score : ')
print(f1_score(y_test,pred_labels))
print('Classification report : ')
print(classification_report(y_test,pred_labels))
print('Confusion matrix : ')
print(confusion_matrix(y_test,pred_labels))


F1 score : 
0.7119507535555085
Classification report : 
              precision    recall  f1-score   support

           0       0.90      0.94      0.92      8196
           1       0.78      0.66      0.71      2550

    accuracy                           0.87     10746
   macro avg       0.84      0.80      0.82     10746
weighted avg       0.87      0.87      0.87     10746

Confusion matrix : 
[[7712  484]
 [ 873 1677]]


#### Some estimators have predict_proba method as well

In [20]:
y_preds = pipeline.predict_proba(X_test)
print(y_preds)

[[0.99165812 0.00834188]
 [0.60663989 0.39336011]
 [0.33310401 0.66689599]
 ...
 [0.72299323 0.27700677]
 [0.70300586 0.29699414]
 [0.98265809 0.01734191]]


#### Part 2:  Use BestModel as a standalone estimator 

In [21]:
X_train, X_test, y_train, y_test =train_test_split(X, y, test_size=0.33, random_state=42)


#### Transform features to make them ready for model input

In [22]:
transform_pipeline =Pipeline([
            # int missing values imputers
            ('intimputer', MeanMedianImputer(
                imputation_method='median', variables=int_cols)),
            # category missing values imputers
            ('catimputer', CategoricalImputer(variables=cat_cols)),
            #
            ('catencoder', OrdinalEncoder()),
            # classification model

 ])

#### Transform X_train and X_test

In [23]:
X_train=transform_pipeline.fit_transform(X_train,y_train)
X_test=transform_pipeline.transform(X_test)


#### Train model and predict

In [24]:
obj.fit(X_train,y_train)
y_pred = obj.predict(X_test)

[I 2023-07-02 12:51:28,455] A new study created in memory with name: example of optuna optimizer
[I 2023-07-02 12:51:29,415] Trial 0 finished with value: 0.8578221695217014 and parameters: {'boosting_type': 'gbdt', 'max_depth': 13, 'learning_rate': 0.034801052770746674, 'random_state': 42}. Best is trial 0 with value: 0.8578221695217014.
[I 2023-07-02 12:51:30,141] Trial 1 finished with value: 0.8629886341902516 and parameters: {'boosting_type': 'gbdt', 'max_depth': 12, 'learning_rate': 0.05509020467459921, 'random_state': 42}. Best is trial 1 with value: 0.8629886341902516.
[I 2023-07-02 12:51:31,062] Trial 2 finished with value: 0.862943951793937 and parameters: {'boosting_type': 'gbdt', 'max_depth': 11, 'learning_rate': 0.052136012208795206, 'random_state': 42}. Best is trial 1 with value: 0.8629886341902516.
[I 2023-07-02 12:51:31,711] Trial 3 finished with value: 0.8466313668335873 and parameters: {'boosting_type': 'gbdt', 'max_depth': 7, 'learning_rate': 0.017766415568758395, 'ra

2023-07-02 12:51:46,046 :: dev :: If refit is set to True, the optimal model will be refit on the entire dataset, i.e., X_train and y_train!
2023-07-02 12:51:46,046 :: dev :: If refit is set to True, the optimal model will be refit on the entire dataset, i.e., X_train and y_train!


#### Check performance of the model

In [25]:
f1_plus_tn
print('F1 score plus TN : ')
print(f1_plus_tn(y_test,pred_labels))
print('F1 score : ')
print(f1_score(y_test,pred_labels))
print('Classification report : ')
print(classification_report(y_test,pred_labels))
print('Confusion matrix : ')
print(confusion_matrix(y_test,pred_labels))

F1 score plus TN : 
7712.711950753555
F1 score : 
0.7119507535555085
Classification report : 
              precision    recall  f1-score   support

           0       0.90      0.94      0.92      8196
           1       0.78      0.66      0.71      2550

    accuracy                           0.87     10746
   macro avg       0.84      0.80      0.82     10746
weighted avg       0.87      0.87      0.87     10746

Confusion matrix : 
[[7712  484]
 [ 873 1677]]


In [26]:
obj.get_best_estimator()

LGBMClassifier(learning_rate=0.0858298629387847, max_depth=10, random_state=42)

In [27]:
obj.best_estimator

LGBMClassifier(learning_rate=0.0858298629387847, max_depth=10, random_state=42)

#### Get fitted randomized search object and its attributes

In [28]:
OptunaObj = obj.get_optimized_object()
print(OptunaObj)

FrozenTrial(number=17, state=TrialState.COMPLETE, values=[0.8638086335265922], datetime_start=datetime.datetime(2023, 7, 2, 12, 51, 43, 29516), datetime_complete=datetime.datetime(2023, 7, 2, 12, 51, 43, 753917), params={'boosting_type': 'gbdt', 'max_depth': 10, 'learning_rate': 0.0858298629387847, 'random_state': 42}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'boosting_type': CategoricalDistribution(choices=('gbdt',)), 'max_depth': IntDistribution(high=15, log=False, low=6, step=1), 'learning_rate': FloatDistribution(high=0.1, log=False, low=0.01, step=None), 'random_state': IntDistribution(high=42, log=False, low=42, step=1)}, trial_id=17, value=None)
